In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# MLP
In this notebook we demonstrate how the catastrophic forgetting heavily affect MLPs.
We train an MLP on a sorted MNIST. In the evaluation phase, the MLPs respond with the last seen digits.

In [13]:
class MLP(nn.Module):
    def __init__(self, checkpoint: str | None = None):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 28*28)
        self.fc2 = nn.Linear(28*28, 256)
        self.fc3 = nn.Linear(256, 10)
        if checkpoint:
            self.load_state_dict(torch.load(checkpoint))
            self.eval()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [43]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# Train set. Here we sort the MNIST by digits and disable data shuffling.
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
sorted_indices = sorted(range(len(train_dataset)//1), key=lambda idx: train_dataset.targets[idx])
train_dataset = torch.utils.data.Subset(train_dataset, sorted_indices)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
# Test set
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [29]:
# Training the model
def train(model, checkpoint, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-6)
    for epoch in range(epochs):
        model.train()
        with tqdm(train_loader) as pbar:
            for images, labels in pbar:
                outputs = model(images.to(device))
                loss = criterion(outputs, labels.to(device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}')
            torch.save(model.state_dict(), checkpoint)

In [44]:
# Loading model from checkpoint
# model = MLP(checkpoint='checkpoint/mlp_mnist_no_shuffle.pth')
model = MLP().to(device)
train(model, 'checkpoint/mlp_mnist_no_shuffle_low_lr.pth',epochs=5)

100%|██████████| 938/938 [00:14<00:00, 63.28it/s]


Epoch 1, Loss: 2.429697275161743


100%|██████████| 938/938 [00:14<00:00, 66.56it/s]


Epoch 2, Loss: 2.3305001258850098


100%|██████████| 938/938 [00:14<00:00, 64.72it/s]


Epoch 3, Loss: 2.2304418087005615


100%|██████████| 938/938 [00:14<00:00, 64.96it/s]


Epoch 4, Loss: 2.138221025466919


100%|██████████| 938/938 [00:14<00:00, 66.62it/s]

Epoch 5, Loss: 2.0530848503112793


In [45]:
# Evaluating
correct = 0
total = 0
predictions = [0] * 10
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images.to(device))
        labels=labels.to(device)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        for pred in predicted:
            predictions[pred] += 1
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')
print(predictions)
print(sum(predictions))

Accuracy: 74.17%
[773, 1249, 677, 1036, 983, 460, 1050, 1441, 1273, 1058]
10000
